### Collect Capacity Data from RPT Folders

 NOTE: The raw data containing Benchmark_Tests_Batch{1-5} should be placed in the root directory. 

In [ ]:
import sys
from pathlib import Path
import os

repo_root = Path.cwd()
while repo_root.name != "Coulombic-Efficiency-Driven-Optimization-of-Health-Aware-Charging-Protocols":
    repo_root = repo_root.parent
sys.path.insert(0, str(repo_root))

from utils.benchmark_capacity import collect_benchmark_rpt_capacities

base_path = os.path.join(os.getcwd(), "data") 

nominal_capacity = 47 / 1000  # Ah

batches = {
    "Benchmark_Tests_Batch1": {
        "cell_groups": {
            "CE Threshold":      [(3, j) for j in range(5, 7)], 
            "CE Lin-Surrogate":  [(4, j) for j in range(5, 9)],
            "CE GP-Surrogate":   [(5, j) for j in range(5, 9)],
            "Constant 3C":       [(6, j) for j in range(5, 9)],
        }
    },
    "Benchmark_Tests_Batch2": {
        "cell_groups": {
            "PBM": [(2, j) for j in range(5, 9)],
        }
    },
    "Benchmark_Tests_Batch3": {
        "cell_groups": {
            "Constant 1C": [(4, j) for j in range(5, 9)],
            "Constant 2C": [(5, j) for j in range(5, 9)],
        }
    },
    "Benchmark_Tests_Batch4": {
        "cell_groups": {
            "CE Lin-Surrogate (Non-Adaptive)": [(2, j) for j in range(5, 9)],
            "CE Threshold (Non-Adaptive)":     [(6, j) for j in range(5, 9)],
        }
    },
    "Benchmark_Tests_Batch5": {
        "cell_groups": {
            "CE Threshold": [(3, j) for j in range(5, 7)],  
        }
    },
}

results_df = collect_benchmark_rpt_capacities(
    base_path=base_path,
    batches=batches,
    nominal_capacity_ah=nominal_capacity,
)

print(results_df.shape)
results_df.head()


### Plotting Capacity Trajectories

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["axes.grid"] = False 

nominal_capacity_Ah = 47 / 1000
eol_ah = 0.70 * nominal_capacity_Ah

adaptive_style = "-"
constant_style = ":"

def base_profile(p):
    return "PBM" if p.startswith("PBM") else p.split(" (")[0]

profiles = list(results_df["Profile"].unique())
constant_profiles = [p for p in profiles if "Constant" in p]

color_map = {
    "CE Threshold": "#006400",
    "CE Lin-Surrogate": "#1723D4",
    "CE GP-Surrogate": "#6D2A81",
    "PBM": "#F39D1B",
    "Constant 1C": "#D62728",
    "Constant 2C": "#D62728",
    "Constant 3C": "#D62728",
    "CE Threshold (Non-Adaptive)": "#727473",
    "CE Lin-Surrogate (Non-Adaptive)": "#727473",
}

marker_map = {
    "CE Threshold": "o",
    "CE Threshold (2nd Run)": "o",
    "CE Threshold (Non-Adaptive)": "o",
    "CE Lin-Surrogate (Non-Adaptive)": "s",
    "CE Lin-Surrogate": "s",
    "CE GP-Surrogate": "D",
    "PBM": "X",
    "Constant 1C": "v",
    "Constant 2C": "<",
    "Constant 3C": ">",
}

legend_order = [
    "CE Threshold",
    "CE Threshold (2nd Run)",
    "CE Lin-Surrogate",
    "CE GP-Surrogate",
    "PBM",
    "CE Threshold (Non-Adaptive)",
    "CE Lin-Surrogate (Non-Adaptive)",
    "Constant 1C",
    "Constant 2C",
    "Constant 3C",
]

fig, ax1 = plt.subplots(figsize=(9, 4), dpi=300, constrained_layout=True)

for prof in profiles:
    bp = base_profile(prof)
    color = color_map.get(prof, "black")
    mk = marker_map[bp]
    ls = constant_style if prof in constant_profiles else adaptive_style

    mean_caps = (
        results_df[results_df["Profile"] == prof]
        .groupby("RPT")["Capacity (Ah)"]
        .mean()
    )

    below = mean_caps[mean_caps < eol_ah]
    if not below.empty:
        cutoff = below.index.min()
        mask = mean_caps.index <= cutoff
    else:
        mask = np.ones(len(mean_caps), dtype=bool)

    x = mean_caps.index[mask]
    y = mean_caps.values[mask]

    ax1.plot(
        x, y,
        linestyle=ls,
        color=color,
        linewidth=3,
        marker=mk,
        markevery=[len(x) - 1],
        markeredgecolor=color,
        markerfacecolor="white",
        markersize=9.5,
        label=prof,
    )

ax1.axhline(eol_ah, color="black", linestyle="--", linewidth=2)
ax1.text(73, eol_ah + 0.0005, "EOL", color="black", fontsize=12, va="bottom")

ax1.set_xlabel("RPT Number (×4 Cycles)", fontsize=14)
ax1.set_ylabel("Capacity (Ah)", fontsize=14)
ax1.set_xlim(0, 81)
ax1.set_xticks(np.arange(0, 81, 10))
ax1.set_ylim(0.03, 0.050)
ax1.tick_params(labelsize=12)

ax2 = ax1.secondary_yaxis(
    "right",
    functions=(lambda x: x / nominal_capacity_Ah * 100,
               lambda x: x * nominal_capacity_Ah / 100),
)
ax2.set_ylabel("Capacity (%)", fontsize=14)
ax2.set_yticks([50, 60, 70, 80, 90, 100])
ax2.tick_params(labelsize=12)

handles, labels = ax1.get_legend_handles_labels()
by_label = dict(zip(labels, handles))

display_label_map = {}
for lab in labels:
    if lab.startswith("Constant "):
        display_label_map[lab] = lab + " CCCV"
    else:
        display_label_map[lab] = lab

ordered = [(lab, by_label[lab]) for lab in legend_order if lab in by_label]
disp_labs, hnds = zip(*[(display_label_map[lab], h) for lab, h in ordered])

ax1.legend(
    hnds, disp_labs,
    fontsize=9,
    loc="upper left",
    bbox_to_anchor=(1.25, 1),
    borderaxespad=0,
    frameon=True,
    edgecolor="black",
)

plt.savefig(
    # "Journal Paper/Raw Exported Figures/Capacity_trajectories_benchmark.svg",
    format="svg",
    dpi=300,
    bbox_inches="tight",
)
plt.show()


### Compute Metrics for Charge Speed and Lifetime Benchmarking

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils.benchmark_speed_metrics import compute_benchmark_speed_metrics

base_dir = os.path.join(os.getcwd(), "data") 

batches = {
    "Benchmark_Tests_Batch1": {
        "cell_groups": {
            'CE Threshold':             [(3, j) for j in range(5, 7)],
            'CE Lin-Surrogate':         [(4, j) for j in range(5, 9)],
            'CE GP-Surrogate':          [(5, j) for j in range(5, 9)],
            'Constant 3C':              [(6, j) for j in range(5, 9)],
        },
        "use_step13_for_rpt0": True,
    },
    "Benchmark_Tests_Batch2": {
        "cell_groups": {'PBM': [(2, j) for j in range(5, 9)]},
        "use_step13_for_rpt0": False,
    },
    "Benchmark_Tests_Batch3": {
        "cell_groups": {
            'Constant 1C': [(4, j) for j in range(5, 9)],
            'Constant 2C': [(5, j) for j in range(5, 9)],
        },
        "use_step13_for_rpt0": False,
    },
    "Benchmark_Tests_Batch4": {
        "cell_groups": {
            'CE Lin-Surrogate (Non-Adaptive)':  [(2, j) for j in range(5, 9)],
            'CE Threshold (Non-Adaptive)':      [(6, j) for j in range(5, 9)],
        },
        "use_step13_for_rpt0": False,
    },
    "Benchmark_Tests_Batch5": {
        "cell_groups": {'CE Threshold': [(3, j) for j in range(5, 7)]},
        "use_step13_for_rpt0": False,
    },
}

out = compute_benchmark_speed_metrics(
    base_dir=base_dir,
    batches=batches,
    nominal_capacity_ah=47/1000,
    eol_pct=70.0,
)

metrics_combined = out["metrics_combined"].copy()

### Plotting Charge Speed and Lifetime Trade-Off  

In [ ]:
# -------------------------
# 1) Prepare data
# -------------------------
plot_df = metrics_combined.copy()

protocol_order = [
    'CE Threshold',
    'CE Lin-Surrogate',
    'CE GP-Surrogate',
    'PBM',
    'CE Threshold (Non-Adaptive)',
    'CE Lin-Surrogate (Non-Adaptive)',
    'Constant 1C',
    'Constant 2C',
    'Constant 3C'
]

plot_df = (
    plot_df
    .groupby('Protocol', as_index=False)
    .agg(
        ChargeTimeMean_s=('ChargeTimeMean_s', 'mean'),
        LifeMean_cycles=('LifeMean_cycles', 'mean'),
    )
)

plot_df = (
    plot_df
    .set_index('Protocol')
    .loc[protocol_order]
    .reset_index()
)

plot_df['SpeedMean'] = 1.0 / plot_df['ChargeTimeMean_s']
plot_df['LifetimeMean'] = plot_df['LifeMean_cycles']

# -------------------------
# 2) Styling
# -------------------------
sns.set_style("white")
plt.rcParams['axes.grid'] = False

color_map = {
    'CE Threshold':   '#006400',
    'CE Lin-Surrogate': "#1723D4",
    'CE GP-Surrogate': "#6D2A81",
    'PBM':            "#F39D1B",
    'Constant 1C':    "#D62728",
    'Constant 2C':    "#D62728",
    'Constant 3C':    "#D62728",
    'CE Threshold (Non-Adaptive)':        "#727473",
    'CE Lin-Surrogate (Non-Adaptive)':    "#727473",
}

marker_map = {
    'CE Threshold':                    'o',
    'CE Threshold (Non-Adaptive)':     'o',
    'CE Lin-Surrogate (Non-Adaptive)': 's',
    'CE Lin-Surrogate':                's',
    'CE GP-Surrogate':                 'D',
    'PBM':                             'X',
    'Constant 1C':                     'v',
    'Constant 2C':                     '<',
    'Constant 3C':                     '>',
}

# -------------------------
# 3) Plot
# -------------------------
fig, ax = plt.subplots(figsize=(9, 4), dpi=300, constrained_layout=True)

for prot in protocol_order:
    row = plot_df[plot_df['Protocol'] == prot].iloc[0]
    ax.scatter(
        row['LifetimeMean'], row['SpeedMean'],
        marker=marker_map[prot], s=250,
        facecolors='white', edgecolors=color_map[prot], linewidths=1.5,
        label=prot
    )

min_speed = plot_df['SpeedMean'].min()
max_speed = plot_df['SpeedMean'].max()

ax.set_xlabel("Lifetime (Cycles to 70 % EOL)", fontsize=14)
ax.set_ylabel(r"Charging Speed (Cycles s$^{-1}$)", fontsize=14)
ax.set_xlim(0, 250)
ax.set_xticks(np.arange(0, 251, 50))

ymin_plot = max(min_speed * 0.01, 1e-8)
ymax_plot = max_speed * 1.3
ax.set_ylim(ymin_plot, ymax_plot)
ax.set_yticks(np.linspace(ymin_plot, ymax_plot, 6))
ax.tick_params(labelsize=12)

# Trade-off lines based on constants
x0, x1 = ax.get_xlim()
baseline_defs = {
    "Constant 1C": dict(ls="--", color="#B2182B", label="1C Trade-Off", alpha=0.5),
    "Constant 2C": dict(ls="--", color="#B2182B", label="2C Trade-Off", alpha=0.75),
    "Constant 3C": dict(ls="--", color="#B2182B", label="3C Trade-Off", alpha=1),
}

baseline_labels = []
for prot in ["Constant 1C", "Constant 2C", "Constant 3C"]:
    if prot in plot_df['Protocol'].values:
        base = plot_df[plot_df['Protocol'] == prot].iloc[0]
        slope = base['SpeedMean'] / base['LifetimeMean']
        if slope <= 0:
            continue

        style = baseline_defs[prot]
        x_max_line = min(x1, ymax_plot / slope)
        x_line = np.array([x0, x_max_line])
        y_line = slope * x_line

        ax.plot(
            x_line, y_line,
            style["ls"], color=style["color"],
            linewidth=2.5, alpha=style["alpha"],
            label=style["label"], zorder=0
        )
        baseline_labels.append(style["label"])

# Legend
handles, labels = ax.get_legend_handles_labels()
legend_order = protocol_order + baseline_labels

ordered = [(lab, handles[labels.index(lab)]) for lab in legend_order if lab in labels]
labs, hnds = zip(*ordered)

ax.legend(
    hnds, labs,
    fontsize=9,
    frameon=True, edgecolor='black',
    loc='upper left', ncol=1,
    markerscale=0.6,
    bbox_to_anchor=(1.25, 1.0),
)

# Secondary y-axis = time(s)
ax2 = ax.secondary_yaxis('right', functions=(lambda s: 1.0/s, lambda t: 1.0/t))
ax2.set_ylabel("Charging Time (s)", fontsize=14)
ax2.set_yticks([800, 1200, 2500, 40000])
ax2.tick_params(labelsize=12, length=0)

plt.tight_layout()
plt.savefig(
    # "Journal Paper/Raw Exported Figures/Chg_speed_vs_lifetime_benchmark.svg",
    format='svg', dpi=300, bbox_inches='tight'
)
plt.show()